# DDPM for 2 dimensional data space
## - DDPM: Denoising Diffusion Probabilistic Model

- 목표: 2 차원 데이터를 사용하여 DDPM의 Diffusion Process 개념을 이해해 본다.

- Reference
 * Code : [1-d DDPM](https://github.com/stefanoscotta/1-d-generative-diffusion-model)
 * Paper: [Understanding and contextualizing diffusion models, 2023](https://arxiv.org/pdf/2302.01394)
 * [Annotated-diffusion](https://huggingface.co/blog/annotated-diffusion)


### Prerequisite
- install seaborn tqdm
```
!pip install seaborn  
!pip install tqdm
```

### 순서 sketch
1. Data Preparation: X0 (Nx2), N x0's 
  - 원 x0의 2차원 데이터 분포로부터 sample N개를 준비한다.
  - 여기서 x0의 2차원 데이터 분포는 Gaussian 분포을 사용한다. 
  
2. DDPM - forward noising process
  - noise scheduler: T steps, beta(1), beta(T) --> beta(t)
    * alpha(t), alpha_bar(t)
    * visualize beta(t), alpha(t), alpha_bar(t)
    * check: SNR(T), alpha_bar(T)
  - x0 sampling: x0 ~ q(x0) (원본 데이터의 확률 분포)
  - forward noising process for given x0: q(x(t)|x0)
    * samples for training
    * visualize : trajectory w/ animation
       - xt_mean
    * x(T) distribution: N(0,1**2)

(참조) 3. DDPM - reverse denoising proces: p_theta(x(t-1)|x(t))
  - models to be trained, train samples 
  - q(x(t-1)|x(t), x0) ~ p(x(t-1)|x(t))
  - Loss, Network design(UNet)
    * mean
    * eps

### dataframe 사용한 data 저장 및 처리
1. X0: N x 2 (n=0,...,N-1)
   - x0: 2 x 1 (or 1x2)
   - N개 x0 (2차원 데이터)
2. Noise scheduler (beta_t, alpha_t, alpha_bar_t): Tx1
   - t: (0,) 1,...,T (T+1)
   - df_ns (noise scheduler) [t] = [beta, alpha, alpha_bar, sqrt_alpha_bar]
3. Xt: N x 2 
   - xt: 2 x 1 (or 1x2)
   - X0, X1,...,XT: (T+1) x (N x 2)
   - x(n, t, d=2): N x (T+1) x 2
     df_Xt [n, t, d] = [xt]

In [ ]:
import numpy as np
from numpy import random
import random as rd

import matplotlib.pyplot as plt
# animation
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation

import seaborn as sns
from tqdm import tqdm

# Gaussian Mixture Models
from sklearn import mixture
# to generate isotropic Gaussian blobs 
from sklearn.datasets import make_blobs
# handling data using pandas
import pandas as pd

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

## 1. Data Generation 
### 1) 2d Data Generation using Gaussian Mixture Models (GMM)

2차원 공간상의 2개의 gaussian 분포가 있는 데이터를 만듭니다.
- (x1, x2)

In [ ]:
n_centers = 2
means =[[-5,-5],[5,5]] # 중심 위치(mean) 2개 
cluster_std = [0.4, 0.5] # 각각의 표준 편차
sample_size = [100, 200] # 각 gaussian의 sample의 갯수

# gaussian blobs with labels
X, y_true = make_blobs(n_samples=sample_size, 
                       centers=means, cluster_std=cluster_std, # 2d means, standard deviations 
                       n_features=2, # 2 dimensional data
                       random_state=0)

In [ ]:
X.shape, X.dtype, y_true.shape, y_true.dtype

### 2) Visualize the generated 2d data


In [ ]:
plt.scatter(X[:,0], X[:,1], c=y_true)

In [ ]:
# np to dataframe df_X0
df_X0= pd.DataFrame({'d1':X[:,0], 'd2':X[:,1]})
df_X0.head()

In [ ]:
# plot x0 distribution
sns.histplot(df_X0, x='d1', y='d2', stat="density", bins=100)


In [ ]:
# the np data to pytorch tensor X0 (torch.tensor, gpu)
X0 = torch.tensor(X).to(torch.float32).to(device)      # to torch 
N = X0.shape[0]
print(X0.shape, N) 

## 2. DDPM - forward noising process

### 1) Noise Scheduling
- DDPM의 beta-linear noise scheduler에 따라 noise scheduling을 한다
- 입력: $T, \beta_1, \beta_T$ 
- 출력: $\beta_t, \alpha_t, \overline \alpha_t$ for any $t \in \{1,\dots, T\}$
- DDPM 예: T=1000, $\beta_1=0.0001, \beta_T=0.02$
- (특히 작은 dataset일 경우) 실험적으로 잘 선정하여야 함. (T는 무한대가 아님)

In [ ]:
def linear_beta_noise_scheduler(timesteps=1000, beta_start=0.0001, beta_end=0.02, device='cuda'):
    # noise schedule
    # the betas grows linearly from beta1 to betaT
    # beta(t) = (beta_T - beta_1)/(T-1) * (t-1) + beta_1, t = 1:T

    betas = torch.linspace(beta_start, beta_end, timesteps, dtype= torch.float32).to(device) 
    alphas = 1 - betas 
    alpha_bars = torch.cumprod(alphas, axis=0).to(torch.float32).to(device)
    
    return betas, alphas, alpha_bars

In [ ]:
# noise scheduler: linear-beta 
# - T=1000, 0.0001, 0.02 (DDPM)
# - T=1000 steps, 

# DDPM - original setting
T = 1000; beta_1 = .0001; beta_T = .02

# parameters
# T = 500; beta_1 = .0004;  beta_T = .06

# for simple test
#T = 10; beta_1 = .02; beta_T = .3

# noise schedule
betas, alphas, alpha_bars = linear_beta_noise_scheduler(timesteps=T, beta_start=beta_1, beta_end=beta_T)

In [ ]:
betas[0], betas[-1], betas.shape

In [ ]:
t=np.linspace(1,T,T, endpoint=True)
fig = plt.figure(figsize = (13, 5))
ax = fig.subplots(1, 2)
ax1 = ax[0]
ax2 = ax[1]
ax1.plot(t, betas.cpu(), label=r'$\beta$') # raw string for Latex formatted display
ax1.plot(t, alphas.cpu(), label=r'$\alpha$')
ax1.plot(t, alpha_bars.cpu(), label=r'$\bar{\alpha}$')
ax1.plot(t, np.sqrt(alpha_bars.cpu()), label=r'$\sqrt{\bar{\alpha}}$')
ax1.plot(t, np.sqrt(1-alpha_bars.cpu()), label=r'$\sqrt{1-\bar{\alpha}}$')
ax1.set_xlabel('t')
ax1.legend(loc='best')

ax2.plot(t, betas.cpu(), label=r'$\beta$')
ax2.set_xlabel('t')
ax2.set_ylabel(r'$\beta$')
ax2.set_title(r'$\beta_t$')


In [ ]:
signal_c = np.sqrt(alpha_bars.cpu())
noise_c = np.sqrt(1-alpha_bars.cpu())
signal_c[-5:],noise_c[-5:]

In [ ]:
# SNR도 정의하고 그려본다
SNR = np.sqrt(alpha_bars.cpu())/np.sqrt(1-alpha_bars.cpu())
plt.loglog(t, SNR)
plt.title(f'SNR(SNR(T)={SNR[-1]:.5f} ~? 0)')

### 2) Forward "noising" process

### 1. q_forward_conditional 함수를 정의한다.

- q_forward_conditional은 q 분포에 의해 주어진 $x_{t-1}$에서 $x_t$의 확률 분포
- $x_t$ ~ $ q(x_t|x_{t-1}) = N(\mu , \sigma^2)$, where $\mu = \sqrt{1- \beta_t} x_{t-1} , \sigma = \sqrt{ \beta_t}$ 
  * $0 < \beta_1 < \beta_2 < ... < \beta_T < 1$
- 표준 정규 분포를 이용하여 $ q(x_t|x_{t-1})$에서 표본을 추출하는 식: $x_t=\sqrt{1 - \beta_t} x_t + \sqrt{\beta_t}\epsilon$ 
  * $\epsilon$ ~ $N(0 , 1^2)$ (표준 정규 분포)

In [ ]:
# generate xt seq using conditional distribution with beta parameter
# x_0 : x0,  with (d1, d2): 2-dimentional data
# return x_t: T+1 sequence (T+1) x 2)
# q_forward_beta
def q_forward_cond(x_0, betas, device):
    t_len = betas.shape[0]
    xt_seq = torch.zeros((t_len+1, 2)).to(device)  # (T+1)x2
    eps = torch.normal(mean=0, std=1, size=(T,2)).to(device) # Tx2, N(0,1) noise
    
    xt_seq[0,:] = x_0
    for t in range(T):
        mean = torch.sqrt(1-betas[t]).to(device)*xt_seq[t] # 2d mean
        noise = torch.sqrt(betas[t]) * eps[t,:] # 2d covariance 
        xt_seq[t+1, :] = mean + noise
    return xt_seq

In [ ]:
# 임의의 index에 있는 x0를 선택하여 conditional probability에 따라 forward noising process T-step 수행 
x0_idx = 1
Xt_b = q_forward_cond(X0[x0_idx,:], betas, device)
Xt_b_np = Xt_b.cpu().detach().numpy()

In [ ]:
plt.plot(Xt_b_np[:,0], Xt_b_np[:,1],'b-')

In [ ]:
%matplotlib notebook
%matplotlib notebook

#%matplotlib inline
#%matplotlib inline

In [ ]:
# animation
# num: an input frame number
def animate_xt_seq_update(num, ax_, Q, xt_seq, title):
    ax_.set_title(f'f{num}')
    if num >=1:    
        tt = num - 1
        uu = xt_seq[1:num+1,0] - xt_seq[:num,0]
        vv = xt_seq[1:num+1,1] - xt_seq[:num,1]
        Q.set_UVC = ax_.quiver(xt_seq[:num,0], xt_seq[:num,1], uu, vv, color='green', scale_units='xy',scale=1, angles='xy')
        ax_.set_title(f'[{title}]t{num}/{xt_seq.shape[0]}: at ({xt_seq[num-1,0]:.2f}, {xt_seq[num-1,1]:.2f}), {len(uu)}')   
    elif num == 0:
        ax_.cla()
        ax_.plot(xt_seq[:,0], xt_seq[:,1],'b-.')
        uu = xt_seq[1:,0] - xt_seq[:-1,0]
        vv = xt_seq[1:,1] - xt_seq[:-1,1]
        Q.set_UVC = ax_.quiver(xt_seq[:-1,0], xt_seq[:-1,1], uu, vv, color='gray', scale_units='xy',scale=1, angles='xy')
        ax_.set_title(f'[{title}]t{num}: at ({xt_seq[0,0]:.2f}, {xt_seq[0,1]:.2f}), {len(xt_seq[:-1,0])}')    
    return (Q,)

def plot_xt_seq(xt_seq, bounds, title='DDPM-forword Animation'):
    t_len = xt_seq.shape[0]-1
    X = xt_seq[:-1,0] 
    Y = xt_seq[:-1,1] 
    u = xt_seq[1:,0] - xt_seq[:-1,0]
    v = xt_seq[1:,1] - xt_seq[:-1,1]
    fig, ax = plt.subplots(figsize = (7, 7))
    ax.plot(xt_seq[:,0], xt_seq[:,1] ,'b-')
    Q = ax.quiver(X, Y, u, v, scale_units='xy',scale=1, angles='xy', color='gray') # animated=False
    if title:
        ax.set_title(title)
    ax.axis('equal')
    ax.set_xlim(bounds[0,0], bounds[0,1])
    ax.set_ylim(bounds[1,0], bounds[1,1])

    anim = FuncAnimation(fig, animate_xt_seq_update, fargs=(ax, Q, xt_seq, title), frames=range(0, t_len+1), interval=30, blit=True)

    return anim

In [ ]:
idx = 0
Xt_b = q_forward_cond(X0[idx,:],betas, device)
bounds = np.array([[-10,10],[-10,10]])
Xt_b_np = Xt_b.cpu().detach().numpy(force=True)
anim2d_beta = plot_xt_seq(Xt_b_np, bounds, "DDPM-Forward-1d")

### 2. q_forward 함수를 정의한다. 
- q_forward는 $\overline \alpha$ 파라미터로 표현된 조건부 확률 분포에 의해 주어진 $x_0$에서 $x_t$로의 확률 분포
- $x_t(x_0)$ ~ $ q(x_t|x_0) = N(\mu_t , \sigma_t ^2)$, where $\mu_t = \sqrt{\overline \alpha_t} x_0 , \sigma_t = \sqrt{1-\overline \alpha_t}$
- 정규 확률 분포를 사용하여 데이터 샘플링 식: $x_t =\sqrt{\overline \alpha_t} x_0 + \sqrt{1-\overline \alpha_t}\epsilon$ 
  * where $\epsilon$ ~ $N(0 , 1^2)$
- (참조) q_forward는 q_forward_conditional 함수에서 유도되는 동등한(equivalent) $x_t$ 분포를 표현함.  </br>
  차이점은 q_forwad_conditional는 $x_t$ sampling에 t step 필요한데 반해, q_forward는 1 step으로 sampling 가능함
  

In [ ]:
# x_0 : x0 (1x2 torch tensor, with (d1, d2))
# return x_t: T sequenced data ((T+1) x 2)
def q_forward(x_0, bar_alphas, device):
    t_len = bar_alphas.shape[0]
    xt_seq = torch.zeros((t_len+1, 2)).to(device) 
    means = torch.sqrt(bar_alphas).reshape(-1,1)*x_0.reshape(1,-1)
    eps = torch.distributions.MultivariateNormal(torch.zeros(2), torch.eye(2)).sample([t_len]).to(device) # sampling from normal distribution
    xt_seq[0,:] = x_0
    xt_seq[1:,:] = means + torch.sqrt(1-bar_alphas).reshape(-1,1) * eps
    return xt_seq

# X_0 : N개 x0 (Nx2 torch tensor, with (d1, d2))
# return Xt_sequence: N samples (N x (T+1) x 2)
def q_forward_array(X_0, bar_alphas, device):
    n_len = X_0.shape[0]
    t_len = bar_alphas.shape[0]
    Xt_seq = torch.zeros((n_len, t_len+1, 2))
        
    for n, xx_0 in enumerate(X_0[:,:]):
        Xt_seq[n, :] = q_forward(xx_0, bar_alphas, device)
    return Xt_seq

In [ ]:
idx = 0 # index : 0~(N-1)중의 임의의 index 선택
Xt_a = q_forward(X0[idx,:], alpha_bars, device) # 선택된 x0에 대해 q_forward 로 x_t 샘플 trajectory 

In [ ]:
Xt_a.device, Xt_a.shape

In [ ]:
%matplotlib inline
%matplotlib inline

In [ ]:
# print(Xt_a.shape)
Xt_a_np = Xt_a.detach().cpu().numpy()
plt.plot(Xt_a_np[:,0], Xt_a_np[:,1],'b.') #'b.-'
plt.show()

$q(x_0)$가 시간이 따라 이동하는 분포를 그려보자. 

$q(x_T) \sim \mathcal{N}(0,1)$ 인지 확인해 보자.

In [ ]:
# select n_samples of x0 from x0 array
# and return generated xt[1...T] list for the sampled x0's
# output(return) :
#  - df_x : x0에 대한 xt trajectory
#  - idx_xs : X0에서 랜덤 선택된 x0 index list [0,...,N-1] 의 n_samples 갯수
#  - idx_ts : time-step index [0, ..., T] (0 포함 주의)
def generate_ddpm_forward_samples(X_0, n_samples, bar_alphas, device, timestep=1):
    # X0에서 n_samples개 선택 (ex: 20 개 sample)
    n_len=X_0.shape[0] # 
    t_len=bar_alphas.shape[0]
    
    idx_xs = [rd.randrange(0, n_len-1) for j in range(n_samples)]
    # assign x0 first,
    tt=[0, *range(1,t_len+1,timestep)] # T > 1
    t_samples = len(tt) # T/step # list_bar_alphas[0] + x0
    idx_ts = tt
    idx = np.zeros((2, n_samples * t_samples), dtype=np.int32) # time 0  : 20
    
    xt_data = q_forward_array(X_0[idx_xs,:], alpha_bars, device).cpu().detach().numpy().reshape(-1,2)
        
    for n in range(n_samples):
        idx[0, n * t_samples:(n+1) * t_samples] = np.array(tt, dtype=np.int32) # time index
        idx[1, n * t_samples:(n+1) * t_samples] = np.full((1,t_samples), idx_xs[n]) # x sample index
    
    df_x = pd.DataFrame(xt_data, columns=['d1', 'd2'])
    df_x['t'] = idx[0]
    df_x['x0_idx'] = idx[1]
    
    return df_x, idx_xs, idx_ts

In [ ]:
# T가 길고, sample수가 증가하면 시간이 많이 걸림
# X0에서 n_sample 갯수의 x0를 선택, 이 x0들에 대해 q_forward trajectory를 얻음
# 
df_Xt, idx_xs, idx_ts = generate_ddpm_forward_samples(X0, n_samples=200, bar_alphas=alpha_bars, device=device, timestep=1)

In [ ]:
# x0의 특정 sample 하나를 선택해서 x(t) trajectory plot
idx2 = 1 # selet a random x0 index for plotting
df_Xt[(df_Xt['x0_idx'] == idx_xs[idx2]) & (df_Xt['t'] == 0) ][['d1','d2']] # (d1, d2) for the selected x0

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(10,10))
df_Xt[df_Xt['x0_idx'] == idx_xs[idx2]][['d1','d2','t']].plot.line(x='d1',y='d2', ax=axes[0,0])
df_Xt[df_Xt['x0_idx'] == idx_xs[idx2]][['t','d1','d2']].reset_index().set_index('t')[['d1','d2']].plot.line(ax=axes[0,1])
df_Xt[df_Xt['x0_idx'] == idx_xs[idx2]][['d1','d2','t']].plot.scatter(x='d1',y='d2',c='t', colormap='viridis', ax=axes[1,0])
df_Xt[df_Xt['t'] == T-1][['d1','d2']].plot.scatter(x='d1',y='d2', ax=axes[1,1])

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,7))
df_Xt[['d1','d2','t']].plot.scatter(x='d1',y='d2',c='t', colormap='viridis', ax=axes[0]) # xt 전체에 대해 xt trajectory data plot
df_Xt[df_Xt['x0_idx'] == idx_xs[idx2]][['t','d1','d2']].plot.scatter(x='d1',y='d2',c='t', colormap='viridis', ax=axes[1]) # 특정 선택된 x0에 대한 xt plot

### t에 따른 xt 분포

In [ ]:
# t step에 따른 분포 이동 가시화

fig, axes = plt.subplots(2, 3, sharex=True, figsize=(15,8))
fig.suptitle(r'$x_t$ distribution vs time', fontsize=16)

#t_list = [0, 101, 201, 301, 401, 499] # 특정 보고자 하는 t step list
t_list = [int(x*T) for x in [0, 0.2, 0.4, 0.6, 0.8, 1]] # 일정 간격 t step 선택
t_list = [x if x < T else T for x in t_list]

for i, t_ in enumerate(t_list):
    m = i//3
    n = i%3
    
    axes[m][n].hist2d(df_Xt[df_Xt['t'] == t_]['d1'], df_Xt[df_Xt['t'] == t_]['d2'], bins=(100,100), range=((-6,6),(-6,6)), cmap=plt.cm.viridis )
    axes[m][n].set_xlim([-6, 6])
    axes[m][n].set_ylim([-6, 6])
    axes[m][n].set_title(f't={t_}', fontsize=16)

### Animation: t에 따른 xt의 분포

In [ ]:
%matplotlib notebook
%matplotlib notebook

#%matplotlib inline
#%matplotlib inline

In [ ]:
# num: an input frame number
def update2d_ddpm_forward(num, df_xt, im, ax, title=None):
    #if num < 1:
    #    return (P1, P2)
    if num > 1:
        t_=num
    else:
        t_=1
        
    ax.clear()
    xedges = np.linspace(bounds[0,0], bounds[0,1],100)
    yedges = np.linspace(bounds[1,0], bounds[1,1],100)    
    x = df_xt[df_xt['t'] == t_]['d1']
    y = df_xt[df_xt['t'] == t_]['d2']
    hist, xedges, yedges = np.histogram2d(x, y, [xedges, yedges])
    if len(hist) == 0:
        print (f't_={t_}')
        ax.set_title(f't={t_},hist_N={len(hist)}', fontsize=16)
        return 
    xidx = np.clip(np.digitize(x, xedges)-1, 0, hist.shape[0]-1)
    yidx = np.clip(np.digitize(y, yedges)-1, 0, hist.shape[1]-1)
    c = hist[xidx, yidx]
    im = ax.scatter(x, y, c=c)
    ax.set_xlim([bounds[0,0], bounds[0,1]])
    ax.set_ylim([bounds[1,0], bounds[1,1]])
    ax.set_title(f't={t_},hist_N={len(hist)}', fontsize=16)
    plt.show()
    return im 

def plot_ddpm_forward_anim(df_xt, title, bounds, interval=100, blit=True, repeat=True):
    """
    Plots the solutions along with the objective function's contour plot.

    Parameters:
    ----------
    df_xt (dataframe): np array containing the xt and x0 
    title (str): The title of the plot.
    bounds (numpy array): A 2x2 numpy array containing the lower and upper bounds
                          of the input variables. The first row should contain the
                          bounds for the first input variable, and the second row
                          should contain the bounds for the second input variable.
    interval: interval in animation update 
    blit: update partial region if blit=True
    repeat: repeat display if True, otherwise once display
    Returns:
    -------
    None
    """
        
    # sample input range uniformly at 0.1 increments
    xaxis = np.arange(bounds[0,0], bounds[0,1], 0.1)
    yaxis = np.arange(bounds[1,0], bounds[1,1], 0.1)
    
    # create a mesh from the axis
    # compute targets
    
    fig = plt.figure(figsize = (5, 5))
    ax = fig.subplots(1, 1)
    
    # figure : plot xt vs time
    # create a filled contour plot with 50 levels and jet color scheme
    # plot the sample as black circles
    t_ = 0
    xedges = np.linspace(bounds[0,0], bounds[0,1],100)
    yedges = np.linspace(bounds[1,0], bounds[1,1],100)    
    x = df_xt[df_xt['t'] == t_]['d1']
    y = df_xt[df_xt['t'] == t_]['d2']
    hist, xedges, yedges = np.histogram2d(x, y, [xedges, yedges]) 
    xidx = np.clip(np.digitize(x, xedges)-1, 0, hist.shape[0]-1)
    yidx = np.clip(np.digitize(y, yedges)-1, 0, hist.shape[1]-1)
    c = hist[xidx, yidx]
    im = ax.scatter(x, y, c=c)
    
    ax.set_xlim([bounds[0,0], bounds[0,1]])
    ax.set_ylim([bounds[1,0], bounds[1,1]])
    ax.set_title(f't={t_}', fontsize=16)
    
    # animation
    anim = FuncAnimation(fig=fig, func=update2d_ddpm_forward, fargs=(df_xt, im, ax, title), frames=range(1,501,2), 
                         interval=interval, blit=blit, repeat=repeat)    
    
    # show the plot
    plt.show()
    
    # You must store the created animation in a varaible that lives as long as the animation should run
    return anim

In [ ]:
df_Xt.head()

In [ ]:
# define range for input
# bounds: range for plot 
bounds = np.array([[-10.0, 10.0], [-10.0, 10.0]])
step = 50

# Animation for DDPM-2d 
# to show animation, you must store the returned anim variable
anim_gd = plot_ddpm_forward_anim(df_Xt, "DDPM Forward Trajectory Histogram in 2D", bounds, interval=50, blit=False, repeat=True)

### 특정 t 시점에서 xt 분포 

In [ ]:
t_ = 301
fig = plt.figure(figsize = (7, 7))
ax = fig.subplots(1, 1)
    
xedges = np.linspace(-6,6,100)
yedges = np.linspace(-6,6,100)    
x = df_Xt[df_Xt['t'] == t_]['d1']
y = df_Xt[df_Xt['t'] == t_]['d2']
hist, xedges, yedges = np.histogram2d(x, y, [xedges, yedges], range=[[-6,6],[-6,6]]) 
xidx = np.clip(np.digitize(x, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(y, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
im = ax.scatter(x, y, c=c)

ax.set_xlim([-6, 6])
ax.set_ylim([-6, 6])
ax.set_title(f't={t_}', fontsize=16)

## 참고 - Animation

$p(x_0)$분포의 몇개의 샘플에 대해 DDPM forward trajectory를 그려보자.

어떠한 임의의 분포 $p(x_0)$든, $p(x_0)$에서 시작하여 시간이 감에 따라서 평균이 $0$인 표준편차가 1인 $\mathcal{N}(0,1)$ 정규 분포에서 샘플들된 점들로 된 분포로 수렴해 가는 것을 볼 수 있을 것이다.

We can also plot some trajectories for some numbers of points of the distribution 
We'll see that starting from our arbitrary distribution $p(x_0)$ the trajectories converge to some point symmetric with respect to $0$, exactly as the points distributed as a $\mathcal{N}(0,1)$.

실습 
1. 임의의 x0에서 시간 t가 t=1...T까지 변화함에 따라 x_t_mean 평균 위치 이동 (drift) 궤적 그려보기(2d)
2. 임의의 x0에서 시간 t가 t=1...T까지 변화함에 따라 x_t_mean 평균 위치 이동 (drift) 및 noise의 표준편차 궤적 그려보기(2d)
3. 임의의 x0에서 시간 t에 따라 noise 추가하여 이동한 위치 x_t의 위치 이동 궤적 그려보기(2d)
4. 3.에서 여러 x0에 대한 이동 궤적 그려 보고 살펴보기

## Diffusion Model Configuration 사용
- dm_config 설정후
- init_dm_config 실행

In [ ]:
# dm_config : Diffusion Model Configuration
# init noise scheduler
def init_dm_config(dm_config):
    beta_1 = dm_config['beta_1']
    beta_T = dm_config['beta_T']
    T = dm_config['T']
    
    if dm_config["scheduler"] == 'beta-linear':
        #the betas grows linearly from beta1 to betaT
        betas, alphas, alpha_bars = linear_beta_noise_scheduler(T, beta_1, beta_T)     
        
    else:
        print("beta-linear is used for the noise scheduler")
        betas, alphas, alpha_bars = linear_beta_noise_scheduler(T, beta_1, beta_T)     
    
    dm_config['betas'] = betas
    dm_config['alphas'] = alphas
    dm_config['alpha_bars'] = alpha_bars
    dm_config['t'] = np.concatenate((np.zeros(1, dtype='int'), np.linspace(1, dm_config['T'], dm_config['T'], endpoint=True, dtype='int'))) # (0, 1, 1+t-step, ...)
    

In [ ]:
dm_config_custom = {"beta_1":.0004, "beta_T":.06, "T":500, 't-step':1, "scheduler":"beta-linear"}
dm_config_ddpm = {"beta_1":.0001, "beta_T":.02, "T":1000, 't-step':1, "scheduler":"beta-linear"}
dm_config = dm_config_ddpm
init_dm_config(dm_config)

In [ ]:
dm_config['alpha_bars'].dtype, dm_config['alpha_bars'].shape

In [ ]:
len(dm_config["t"])
tt = dm_config['t'][:-1]
alpha_bar_t = dm_config['alpha_bars'][tt]


In [ ]:
def q_forward_xt_mean_std(x_0, t, bar_alphas, device):
    alpha_bar_t = bar_alphas[t]
    
    xt_mean = torch.sqrt(alpha_bar_t).reshape(alpha_bar_t.shape[0],-1)*x_0.reshape(1,-1)
    xt_std = torch.sqrt(1-alpha_bar_t) # the same 2d covariance
                
    return xt_mean, xt_std

# from single x0, xt_mean trajectory for t=1, ...T

def q_forward_xt_mean_std_array(x_0, device='cpu', **config):
    len_data=x_0.shape[0] # x0_n_samples = 20
    
    T = config['T']
    alpha_bars = config['alpha_bars']
    t_step = config['t-step']
    
    tt=range(0, T, t_step) # T > 1
    t_samples = len(tt)  # T/step # list_bar_alphas[0] + x0
    
    xt_means, xt_stds = q_forward_xt_mean_std(x_0, tt, alpha_bars, device)


    n_samples = 1 # len(idx_ts)
    xt_data = xt_means.cpu().numpy() # 51(t_samples) x 20(n_samples) x 2
    xt_std_data = xt_stds.cpu().numpy() # 51(t_samples) x 20(n_samples) x 2
    
    # to dataframe
    df_x = pd.DataFrame(xt_data)
    df_x.columns=['m1', 'm2']
    df_x['t'] = tt
    df_x['std'] = xt_std_data
    
    return df_x 

In [ ]:
# 임의의 x_0에 대해 t step에서 xt의 평균 구하기 (xt_mean)
x0_idx = 1
df_xt_ms = q_forward_xt_mean_std_array(X0[x0_idx], device, **dm_config)
len(df_xt_ms), df_xt_ms.head()

In [ ]:
%matplotlib inline
%matplotlib inline

In [ ]:
# plot for random x0 index

fig, axes = plt.subplots(1,3, figsize=(12,4))
df_xt_ms[['m1','m2','t']].plot.scatter(x='m1',y='m2',c='t', colormap='viridis', ax=axes[0])
df_xt_ms[['m1','m2','t']].plot.line(x='m1',y='m2', ax=axes[1])
df_xt_ms[['t','m1','m2']].reset_index().set_index('t')[['m1','m2']].plot.line(ax=axes[2])

In [ ]:
# n point에 대해 scatter plot (m1, m2), radius = std
n = 1000
idx = slice(0,n,30)
cmap = plt.get_cmap("viridis")
colors = cmap(df_xt_ms['t']/df_xt_ms['t'].shape[0]*255)
df_xt_ms['c'] = df_xt_ms['t'].apply(lambda x: cmap(x/df_xt_ms['t'].shape[0]*255))
g = plt.scatter(df_xt_ms[['m1']][idx], df_xt_ms[['m2']][idx],s=df_xt_ms[['std']][idx]*1000, marker='o', facecolors='none', edgecolors=df_xt_ms['c'] )
g.set_facecolor('none')
plt.colorbar()

plt.show()

In [ ]:
from IPython.display import display

display(df_xt_ms.head())


In [ ]:
import matplotlib.cm as cm

def plot_dm_x0_trajectory(df_xt, step=50):
    plt.figure()
    ax = plt.gca()

    for index, row in df_xt.iterrows(): 
        if index % step != 0:
            continue
            
        x = row['m1']
        y = row['m2']
        t = row['t']
        c = t/T
        color = cm.viridis(t/T) # color = cm.jet(t/T)
        size = row['std']
        
        if t > T:
            break
        circle=plt.Circle((x, y), size, color=color, fill=False)
        dot=plt.Circle((x, y), 0.02, color=color, fill=True)
        
        ax.add_artist(circle)
        ax.add_artist(dot)

    plt.xlim([-6,7])
    plt.ylim([-6,7])
    ax.set_aspect(1.0)

    if 'm1' in df_xt:
        tt = range(0, len(df_xt), step)
        plt.plot(df_xt[['m1']].loc[tt], df_xt[['m2']].loc[tt], '.-') 

In [ ]:
x_idx = 6
df_xt_ms = q_forward_xt_mean_std_array(X0[x0_idx], device, **dm_config)

plot_dm_x0_trajectory(df_xt_ms, 40)

In [ ]:
display(df_xt_ms.head()), len(df_xt_ms)

# Animation for trajectory

In [ ]:
# x_0 : N개의 x0 (d1, d2)
# return : x0 trajectory [xt] (xt_samples, xt_means, xt_stds)
# note: x0 is not included in returned tensors
def q_forward_trajectory(x_0, t, bar_alphas, device, including_x0=True):
    assert (len(t) <= len(bar_alphas))
    
    alpha_bar_t = bar_alphas[t]
    s = torch.zeros(len(t), x_0.shape[0], device=device)
    
    xt_means = torch.sqrt(alpha_bar_t).reshape(alpha_bar_t.shape[0],-1)*x_0.reshape(1,-1) # dimension
    xt_stds = torch.sqrt(1-alpha_bar_t) # the same 2d covariance
    
    cov = torch.eye(x_0.shape[0]).to(device) # 2d covariance 
    
    for n, t_ in enumerate(t):
        cov2 = cov*(1-alpha_bar_t[n]) # the same 2d covariance
        s[n, :] = torch.distributions.MultivariateNormal(xt_means[n,:], cov2).sample().to(device)

    # if you want to include x0
    if including_x0 == True:
        x_0 = torch.tensor(x_0).unsqueeze(0).to(device)
        s = torch.cat([x_0, s], axis=0)    
        xt_means = torch.cat([x_0, xt_means], axis=0)    
        xt_stds = torch.cat([torch.zeros(1, dtype=torch.float32, device=device), xt_stds], axis=0)    

    return s, xt_means, xt_stds
    

In [ ]:
#  임의로 샘플된 x0(x0_index)에 대한 test
x0_idx = 0 # sample index for x0
t_step = dm_config['t-step']
tt = dm_config['t'][:-1] # 0, ..., T-1 for indexing

xt, xt_means, xt_noises = q_forward_trajectory(X0[x0_idx], tt, dm_config['alpha_bars'], device)

In [ ]:
xt.shape, xt_means.shape, xt_noises.shape, len(dm_config['t'][:-1]) # excluding last index for t

In [ ]:
xt.shape, len(dm_config['t']), xt_means.shape, xt_noises.shape

In [ ]:
df_xt2 = pd.DataFrame({'t':dm_config['t'], 'd1':xt[:,0].cpu().numpy(), 'd2':xt[:,1].cpu().numpy(), 
                       'm1':xt_means[:,0].cpu().numpy(), 'm2':xt_means[:,1].cpu().numpy(), 'std': xt_noises.cpu().numpy()})
df_xt2.head()

In [ ]:
plot_dm_x0_trajectory(df_xt2, 20)

# Animation

In [ ]:
%matplotlib notebook
%matplotlib notebook

In [ ]:
# TODO
# - bound 추가, 
# - 느려짐
# num: an input frame number

In [ ]:
def update2d_line(num, df_xt, P1, P2, lines, ax, title=None):
    if num > 1:
        t_=num
    else:
        t_=1
    print(f'update:{num}, t={t_}')
    T_ = df_xt.shape[0] 
    T = T_
    xt_1 = df_xt[['d1']].loc[t_]
    xt_2 = df_xt[['d2']].loc[t_]
    mt_1 = df_xt[['m1']].loc[t_]
    mt_2 = df_xt[['m2']].loc[t_]
    # figure 1
    P1.set_data(df_xt[['d1']].loc[:t_], df_xt[['d2']].loc[:t_]) # P: line
    
    # figure 2
    size = df_xt[['std']].loc[t_]
    circle=plt.Circle((mt_1, mt_2), size, color='b', fill=False)
    mt_dot=plt.Circle((mt_1, mt_2), 0.02, color='g', fill=True)
    xt_dot=plt.Circle((xt_1, xt_2), 0.01, color='r', fill=True)
        
    ax[0,1].add_artist(circle)
    ax[0,1].add_artist(mt_dot)
    ax[0,1].add_artist(xt_dot)
    
    # figure 3
    if num == 0:
        ax[1,0].cla()
    ax[1,0].plot(df_xt[['d1']].loc[:t_], df_xt[['d2']].loc[:t_], '*', color='b') # P=line 
    
    ax[0,0].set_title(f'[xt]t={t_}/{T}') 
    ax[0,1].set_title(f'[mt]t={t_}/{T}') 
       
    ax[1,0].set_title(f't={t_}/{T}') 
        
    # figure 4 
    lines[0].set_data(df_xt[['d1']].loc[:t_], df_xt[['d2']].loc[:t_] )
    lines[1].set_data(df_xt[['m1']].loc[:t_], df_xt[['m2']].loc[:t_] )
    
    ax[1,1].set_title(f't={t_}/{T}, num={num}') 
    
    return (P1, P2, lines)

def plot_dm_trajectory_anim_line(df_xt, title, bounds, interval=100, blit=True, repeat=True):
    """
    Plots the solutions along with the objective function's contour plot.

    Parameters:
    ----------
    df_xt (dictionary): np array containing the xt and x0 
    title (str): The title of the plot.
    bounds (numpy array): A 2x2 numpy array containing the lower and upper bounds
                          of the input variables. The first row should contain the
                          bounds for the first input variable, and the second row
                          should contain the bounds for the second input variable.
    interval: interval in animation update 
    blit: update partial region if blit=True
    repeat: repeat display if True, otherwise once display
    Returns:
    -------
    None
    """
        
    # sample input range uniformly at 0.1 increments
    xaxis = np.arange(bounds[0,0], bounds[0,1], 0.1)
    yaxis = np.arange(bounds[1,0], bounds[1,1], 0.1)
    
    fig = plt.figure(figsize = (10, 10))
    ax = fig.subplots(2, 2)
    ax1 = ax[0,0]
    ax2 = ax[0,1]
    ax3 = ax[1,0]
    ax4 = ax[1,1]
    
    # figure 1 : plot xt vs time
    P1, = ax1.plot(df_xt[['d1']], df_xt[['d2']], '.-', color='r') # P=line 
    ax1.set_title(f'{title}: xt')
    ax1.set_xlabel('d1')
    ax1.set_ylabel('d2')
    ax1.set_xlim(bounds[0,0], bounds[0,1])
    ax1.set_ylim(bounds[1,0], bounds[1,1])
    
    # figure 2: plot mean_t vs time
    P2, = ax2.plot(df_xt[['m1']], df_xt[['m2']], '.-', color='g') # P=line 
    ax2.cla()
    ax2.set_title(f'{title}: mt')
    ax2.set_xlabel('d1')
    ax2.set_ylabel('d2')
    ax2.set_xlim(bounds[0,0], bounds[0,1])
    ax2.set_ylim(bounds[1,0], bounds[1,1])
    ax2.set_aspect(1.0)

    # figure 3 : scatter plot for xt    
    ax3.plot(df_xt[['d1']], df_xt[['d2']], '*', color='b') # P=line 
    ax3.cla()
    ax3.set_title(f'{title}: xt')
    ax3.set_xlabel('d1')
    ax3.set_ylabel('d2')
    ax3.set_xlim(bounds[0,0], bounds[0,1])
    ax3.set_ylim(bounds[1,0], bounds[1,1])
    
    # figure 4 : x(t)-x(t-1) and mean(t) ~ mean(t-1)
    num = 1 # t= 1
    t = 10
    lines = []
    line, = ax4.plot(df_xt[['d1']][t-1:t], df_xt[['d2']][t-1:t], '.-', color='r' )
    lines.append(line)
    line, = ax4.plot(df_xt[['m1']][t-1:t], df_xt[['m2']][t-1:t], '.-', color='g' )
    lines.append(line)
    
    ax4.set_xlabel('d1')
    ax4.set_ylabel('d2')
    ax4.set_title('xt and mt')
    ax4.set_xlim(bounds[0,0], bounds[0,1])
    ax4.set_ylim(bounds[1,0], bounds[1,1])
    
    # animation
    anim = FuncAnimation(fig=fig, func=update2d_line, fargs=(df_xt, P1, P2, lines, ax, title), frames=range(df_xt.shape[0]), interval=interval, blit=blit, repeat=repeat)    

    # show the plot
    plt.show()
    
    # You must store the created animation in a variable that lives as long as the animation should run
    return anim

In [ ]:
# define range for input
# bounds: range for plot 
bounds = np.array([[-10.0, 10.0], [-10.0, 10.0]])
step = 50

# Animation for DDPM-2d 
# to show animation, you must store the returned anim variable
anim_gd = plot_dm_trajectory_anim_line(df_xt2, "DDPM Trajectory in 2D", bounds, interval=1, blit=True, repeat=True)

In [ ]:
%matplotlib inline
%matplotlib inline